In [1]:
import os
from dotenv import load_dotenv

from github import Github
import re
from datasets import Dataset

C:\Users\malha\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--> Collecting Python Code snippets from Github

In [2]:
load_dotenv()

# Acces your Git Token using Environment Variables
git_token = os.getenv('GIT_TOKEN')

g = Github(git_token)
repo = g.get_repo("openai/gym")

--> Extracting the functions, as they start with 'def'

In [3]:
def extract_functions_from_code(code):
    pattern = re.compile((r"def\s+(\w+)\s*\(.*\):"))
    functions = pattern.findall(code)
    return functions

--> Fetching the .py files

In [4]:
python_files = []

contents = repo.get_contents("")

while contents:
    file_content = contents.pop(0)
    if file_content.type == "dir":
        contents.extend(repo.get_contents(file_content.path))
    elif file_content.path.endswith(".py"):
        python_files.append(file_content)

--> Separating and storing the code and the corresponding function name

In [5]:
data = {"code": [], "function_name": []}

for file in python_files:
    code = file.decoded_content.decode("utf-8")
    functions = extract_functions_from_code(code)
    for function in functions:
        data["code"].append(code)
        data["function_name"].append(function)

--> Creating a Hugging Face Dataset

In [6]:
dataset = Dataset.from_dict(data)

dataset.save_to_disk("code_generation_dataset")

print("The dataset has been created and saved to the disk")

Saving the dataset (1/1 shards): 100%|██████████| 974/974 [00:00<00:00, 98382.91 examples/s] 

The dataset has been created and saved to the disk


--> Using a pre-trained LLM (Salesforce)

In [7]:
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

In [8]:
tokenizer = AutoTokenizer.from_pretrained('Salesforce/codegen-350M-mono')
model = AutoModelForCausalLM.from_pretrained('Salesforce/codegen-350M-mono')

tokenizer.pad_token = tokenizer.eos_token

dataset = load_from_disk("code_generation_dataset")

# Split the dataset
dataset = dataset.train_test_split(test_size=0.1)

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

In [11]:
# Preprocess the dataset

def preprocess_function(examples):
    inputs = tokenizer(examples['code'], truncation=True, padding='max_length')
    # For causal language modeling, the labels are the input tokens shifted by one.
    inputs['labels'] = inputs['input_ids'].copy()
    return inputs

--> Fine-tuning

In [12]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# The actual fine-tuning
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    save_steps=10_000,
    save_total_limit=2
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test']
)

trainer.train()

Map: 100%|██████████| 98/98 [00:00<00:00, 336.68 examples/s]


KeyboardInterrupt: 

--> Saving the model and the progress

In [ ]:
trainer.save_model("./fine_tuned_codegen_model")
tokenizer.save_pretrained("./fine_tuned_codegen_model")

--> Testing and code generation

In [11]:
def generate_code(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs['input_ids'], max_length = max_length),
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return generated_code


In [ ]:
# prompt = "def merger_sort(arr):"
# generated_code = generate_code(prompt)

# print("The generated code is: ")
# print(generated_code)